# Simulate

This example gives a "hello world" example to use AMS.

## Import and Setting the Verbosity Level

We first import the `ams` library.

In [1]:
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2023-11-07 22:30:50
ams:0.7.3.post29.dev0+g23b6565


We can configure the verbosity level for logging (output messages) by passing a verbosity level (10-DEBUG, 20-INFO, 30-WARNING, 40-ERROR, 50-CRITICAL) to the `stream_level` argument of `ams.main.config_logger()`. Verbose level 10 is useful for getting debug output.

The logging level can be altered by calling ``config_logger`` again with new ``stream_level`` and ``file_level``.

In [3]:
ams.config_logger(stream_level=20)

Note that the above ``ams.config_logger()`` is a shorthand to ``ams.main.config_logger()``.

If this step is omitted, the default `INFO` level (`stream_level=20`) will be used.

## Run Simulations

### Load Case

AMS support multiple input file formats, including AMS ``.xlsx`` file, MATPOWER ``.m`` file, PYPOWER ``.py`` file, and PSS/E ``.raw`` file.

Here we use the AMS ``.xlsx`` file as an example. The source file locates at ``$HOME/ams/ams/cases/ieee39/ieee39_uced.xlsx``.

In [4]:
sp = ams.load(ams.get_case('ieee39/ieee39_uced.xlsx'),
              default_config=True,
              setup=True)

Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee39/ieee39_uced.xlsx"...
Input file parsed in 0.1640 seconds.
System set up in 0.0041 seconds.


### Inspect Models and Routines

In AMS, ``model`` refers to the device model, and all models are registered to an OrderedDict ``models``.

In [5]:
sp.models

OrderedDict([('Summary', Summary (3 devices) at 0x1069a7670),
             ('Bus', Bus (39 devices) at 0x161e03850),
             ('PQ', PQ (19 devices) at 0x161e16250),
             ('PV', PV (9 devices) at 0x161e16790),
             ('Slack', Slack (1 device) at 0x161e25220),
             ('Shunt', Shunt (0 devices) at 0x161e25ca0),
             ('Line', Line (46 devices) at 0x161e2e190),
             ('ESD1', ESD1 (0 devices) at 0x161e3c880),
             ('REGCV1', REGCV1 (0 devices) at 0x161e3cfa0),
             ('Area', Area (2 devices) at 0x161e4f730),
             ('Region', Region (2 devices) at 0x161e4feb0),
             ('SFR', SFR (2 devices) at 0x161e5a6a0),
             ('SR', SR (2 devices) at 0x161e5ad00),
             ('NSR', NSR (2 devices) at 0x161e64160),
             ('GCost', GCost (10 devices) at 0x161e64580),
             ('SFRCost', SFRCost (10 devices) at 0x161e64c10),
             ('SRCost', SRCost (10 devices) at 0x161e731f0),
             ('NSRCost', NSRCos

One can inspect the detailed model data by converting it to a pandas DataFrame.

In [6]:
sp.PQ.as_df()

,idx,u,name,bus,Vn,p0,q0,vmax,vmin,owner
uid,,,,,,,,,,
0,PQ_1,1.0,PQ_1,3,345.0,6.000,2.500,1.2,0.8,1
1,PQ_2,1.0,PQ_2,4,345.0,4.500,1.840,1.2,0.8,1
2,PQ_3,1.0,PQ_3,7,345.0,2.338,0.840,1.2,0.8,1
3,PQ_4,1.0,PQ_4,8,345.0,5.220,1.766,1.2,0.8,1
4,PQ_5,1.0,PQ_5,12,138.0,1.200,0.300,1.2,0.8,1
5,PQ_6,1.0,PQ_6,15,345.0,3.200,1.530,1.2,0.8,1
6,PQ_7,1.0,PQ_7,16,345.0,3.290,0.323,1.2,0.8,1
7,PQ_8,1.0,PQ_8,18,345.0,1.580,0.300,1.2,0.8,1
8,PQ_9,1.0,PQ_9,20,138.0,6.800,1.030,1.2,0.8,1


In AMS, all supported routines are registered to an OrderedDict ``routines``.

In [7]:
sp.routines

OrderedDict([('DCPF', DCPF at 0x161e035e0),
             ('PFlow', PFlow at 0x161e89a30),
             ('CPF', CPF at 0x161e9a040),
             ('ACOPF', ACOPF at 0x161e9a640),
             ('DCOPF', DCOPF at 0x161eb0340),
             ('ED', ED at 0x161eb08e0),
             ('ED2', ED2 at 0x161ebfbb0),
             ('RTED', RTED at 0x161ecec70),
             ('RTED2', RTED2 at 0x162008670),
             ('UC', UC at 0x165495a30),
             ('UC2', UC2 at 0x1658735e0),
             ('LDOPF', LDOPF at 0x16587ec40),
             ('LDOPF2', LDOPF2 at 0x16588b730)])

### Solve an Routine

Before solving an routine, we need to initialize it first.
Here Real-time Economic Dispatch (RTED) is used as an example.

In [8]:
sp.RTED.init()

Routine <RTED> initialized in 0.0060 seconds.


True

Then, one can solve it by calling ``run()``.
Here, argument `solver` can be passed to specify the solver to use, such as `solver='GUROBI'`.

Installed solvers can be listed by ``ams.shared.INSTALLED_SOLVERS``,
and more detailes of solver can be found at [CVXPY-Choosing a solver](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver).

In [9]:
sp.RTED.run()

RTED solved as optimal in 0.0118 seconds, converged after 50 iterations using solver OSQP.


True

The solved results are stored in each variable itself.
For example, the solved power generation of ten generators
are stored in ``pg.v``.

In [10]:
sp.RTED.pg.v

array([6.01822025, 5.99427957, 5.97043409, 5.08      , 5.98234496,
       5.8       , 5.64      , 6.00623798, 6.03022646, 6.04225669])

Here, ``get_idx()`` can be used to get the index of a variable.

In [11]:
sp.RTED.pg.get_idx()

['PV_30',
 'PV_31',
 'PV_32',
 'PV_33',
 'PV_34',
 'PV_35',
 'PV_36',
 'PV_37',
 'PV_38',
 'Slack_39']

Part of the solved results can be accessed with given indices.

In [12]:
sp.RTED.get(src='pg', attr='v', idx=['PV_30', 'PV_31'])

array([6.01822025, 5.99427957])

All variables are listed in an OrderedDict ``vars``.

In [13]:
sp.RTED.vars

OrderedDict([('pg', Var: StaticGen.pg),
             ('pn', Var: Bus.pn),
             ('plf', Var: Line.plf),
             ('pru', Var: StaticGen.pru),
             ('prd', Var: StaticGen.prd)])

The objective value can be accessed with ``obj.v``.

In [14]:
sp.RTED.obj.v

592.8203580808987

Similarly, the constraints are listed in an OrderedDict ``constrs``,
and the expression values can also be accessed.

In [15]:
sp.RTED.constrs

OrderedDict([('pb', [ON]: sum(pl) - sum(pg) =0),
             ('pinj', [ON]: CftT@plf - pl - pn =0),
             ('lub', [ON]: PTDF @ (pn - pl) - rate_a <=0),
             ('llb', [ON]: - PTDF @ (pn - pl) - rate_a <=0),
             ('rbu', [ON]: gs @ multiply(ug, pru) - dud =0),
             ('rbd', [ON]: gs @ multiply(ug, prd) - ddd =0),
             ('rru', [ON]: multiply(ug, pg + pru) - pmax <=0),
             ('rrd', [ON]: multiply(ug, -pg + prd) - pmin <=0),
             ('rgu', [ON]: multiply(ug, pg-pg0-R10) <=0),
             ('rgd', [ON]: multiply(ug, -pg+pg0-R10) <=0)])

In [16]:
sp.RTED.rgu.v

array([-99.48177975, -99.50572043, -99.52956591, -98.92      ,
       -99.61765504, -98.8       , -98.76      , -99.49376202,
       -99.46977354, -99.73645331])